# Boosting

- bagging이 여러개의 분류기를 만들었다면 boosting은 약한 분류기를 훈련시켜 강하게 만듬
- 성능은 좋을 수 있으나 일반적으로 bagging(RF)에 비해 많이 오래걸림
- adaboost와 gbm이 대표적이며 gbm을 중점으로 다룰 예정
    - adaboost는 반복해서 분류기를 훈련하면서 가중치를 두어 최종 결과를 만듬
    - gbm은 그 과정을 경사하강법(gradient descent)를 사용함
  
    
- parameters: tree에서 계승 하며 훈련을 위한 추가 파라미터가 존재
    - boosting기법은 파라미터의 최적화가 매우 중요함
    - 여러 파라미터를 어떻게 튜닝할지도 함께 알아보자
- 자체 함수도 있지만 많이 사용하는 xgboost와 lightgbm을 알아보자

# XGBoost
- 시간을 단축하기 위해 병렬 처리를 가능하게 해주는 패키지
- gpu연산도 손쉽게 가능한 것이 장점
- (파이썬 래퍼의 경우)자체 데이터셋을 사용(DMatrix) => 최적화가 잘 되어있다고 함
    - sklearn 래퍼를 사용할 것이므로 알아두기만 하자
- parameters:
    - booster: 사용할 알고리즘 종류
        - gbtree, dart, gblinear
    - nthread: 병렬연산 관련  
    - ----------------
    - n_estimators: 여러번 훈련시키는 그 여러번에 해당
    - learning_rate: 오차를 얼마큼 반영시켜 답을 찾을 지
    - max_depth: 트리 최대 깊이
    - subsample: 데이터를 샘플링하는 비율(과적합방지)
    - colsample_bytree: 변수를 샘플링하는 비율(과적합방지)
    - gamma: 트리를 나누기 위한 손실 감소 기준
    - reg_lamda: L2 규제
    - reg_alpha: L1 규제
    - -----------
    - objective: 학습규칙
        - reg:linear, munlti:softmax 등
    - eval_metric: 모형에 손실을 계산하기 위한 평가 규칙
        - rmse, mae, error, auc 등
        
+ ++ early_stopping_rounds: 조기 중단 시켜 시간을 단축시킬 수 있는 기능을 줌

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = pd.read_csv('../1.clustering/titanic.csv')

target = data['Survived']
data = data.drop(['PassengerId', 'Survived'], axis=1)

def sex(a):
    if a == 'male':
        return 0
    else:
        return 1
    
def emb(a):
    if a == 'S':
        return 0
    elif a == 'Q':
        return 1
    else:
        return 2
    
data['Sex'] = data['Sex'].map(sex)
data['Embarked'] = data['Embarked'].map(emb)

train_X, test_X, train_y, test_y = train_test_split(data, target, test_size=0.3, random_state=2019, shuffle=True)

In [2]:
from xgboost import XGBClassifier

params= {
    'booster': 'gbtree',
    'tree_method': 'gpu_hist',
    'n_estimators': 400,
    'learning_rate': 0.1,
    'max_depth': 6,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
#     'reg_alpha': 1,
#     'reg_lambda': 2
    'random_state': 0
}



In [3]:
xgb_clf = XGBClassifier(**params).fit(train_X, train_y, early_stopping_rounds=100, eval_metric='error', eval_set=[(test_X, test_y)])
print(accuracy_score(test_y, xgb_clf.predict(test_X)))
## eval_metric 설정으로 달라지는 학습결과 보기

[0]	validation_0-error:0.257009
Will train until validation_0-error hasn't improved in 100 rounds.
[1]	validation_0-error:0.224299
[2]	validation_0-error:0.252336
[3]	validation_0-error:0.205607
[4]	validation_0-error:0.219626
[5]	validation_0-error:0.224299
[6]	validation_0-error:0.191589
[7]	validation_0-error:0.21028
[8]	validation_0-error:0.186916
[9]	validation_0-error:0.182243
[10]	validation_0-error:0.200935
[11]	validation_0-error:0.205607
[12]	validation_0-error:0.196262
[13]	validation_0-error:0.205607
[14]	validation_0-error:0.186916
[15]	validation_0-error:0.205607
[16]	validation_0-error:0.200935
[17]	validation_0-error:0.200935
[18]	validation_0-error:0.196262
[19]	validation_0-error:0.200935
[20]	validation_0-error:0.191589
[21]	validation_0-error:0.191589
[22]	validation_0-error:0.196262
[23]	validation_0-error:0.191589
[24]	validation_0-error:0.205607
[25]	validation_0-error:0.219626
[26]	validation_0-error:0.196262
[27]	validation_0-error:0.196262
[28]	validation_0-er

# LightGBM

- xgboost와 양대산맥을 이루는 tree관련 패키지
- xgboost보다 빠른 연산 시간을 보여준다고 함
- 최신 패키지인 만큼 기능도 몇 가지 보안됨
- 가장 큰 특징으로 leaf wise 분할 (반대는 level wise)
    - 이로 인해 보통 트리 깊이가 더 깊어짐
- 데이터 수가 적을 경우 과적합이 더 자주 발생한다고 함 (공식문서에서 만건 이하)
- 파라미터는 거의 유사

In [4]:
from lightgbm import LGBMClassifier

params= {
    'boosting': 'gbdt',
#     'tree_method': 'gpu_hist',
    'n_estimators': 400,
    'learning_rate': 0.1,
    'max_depth': 9,
    'min_child_samples': 20, # 결정노드가 되기 위한 최소 자료 수
    'num_leaves': 31, # 트리가 가질 수 있는 최대 리프 수
    'subsample': 0.7, # bagging_fraction
    'colsample_bytree': 0.7, # feature_fraction
    'random_state': 0,
}

lgb_clf = LGBMClassifier(**params).fit(train_X, train_y, early_stopping_rounds=100,eval_set=[(test_X, test_y)], eval_metric='auc')
print(accuracy_score(test_y, lgb_clf.predict(test_X)))

[1]	valid_0's auc: 0.828899	valid_0's binary_logloss: 0.630457
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.834996	valid_0's binary_logloss: 0.602703
[3]	valid_0's auc: 0.833149	valid_0's binary_logloss: 0.590206
[4]	valid_0's auc: 0.843173	valid_0's binary_logloss: 0.56688
[5]	valid_0's auc: 0.842249	valid_0's binary_logloss: 0.547903
[6]	valid_0's auc: 0.839616	valid_0's binary_logloss: 0.533839
[7]	valid_0's auc: 0.843727	valid_0's binary_logloss: 0.519588
[8]	valid_0's auc: 0.840262	valid_0's binary_logloss: 0.516946
[9]	valid_0's auc: 0.843126	valid_0's binary_logloss: 0.505515
[10]	valid_0's auc: 0.840955	valid_0's binary_logloss: 0.504042
[11]	valid_0's auc: 0.841787	valid_0's binary_logloss: 0.495516
[12]	valid_0's auc: 0.846175	valid_0's binary_logloss: 0.491644
[13]	valid_0's auc: 0.847284	valid_0's binary_logloss: 0.484786
[14]	valid_0's auc: 0.84539	valid_0's binary_logloss: 0.479464
[15]	valid_0's auc: 0.84539	valid_0's binary_logloss:

In [5]:
## customized eval metrics
def custom_eval_metric(y_true, y_hat):
    ## y_true는 실제 값, y_hat은 확률로 넘어와서 비교해야 함
    y_hat = np.where(y_hat < 0.5, 0, 1)
    return 'custom', accuracy_score(y_true, y_hat), True

params= {
    'boosting': 'gbdt',
#     'tree_method': 'gpu_hist',
    'n_estimators': 400,
    'learning_rate': 0.1,
    'max_depth': 9,
    'min_child_samples': 20, # 결정노드가 되기 위한 최소 자료 수
    'num_leaves': 31, # 트리가 가질 수 있는 최대 리프 수
    'subsample': 0.9, # bagging_fraction
    'colsample_bytree': 0.9, # feature_fraction
    'random_state': 0,
    'metric': 'auc'
}

lgb_clf = LGBMClassifier(**params).fit(train_X, train_y, early_stopping_rounds=100,eval_set=[(test_X, test_y)], eval_metric=custom_eval_metric)
print(accuracy_score(test_y, lgb_clf.predict(test_X)))



[1]	valid_0's auc: 0.821508	valid_0's custom: 0.616822
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.826173	valid_0's custom: 0.742991
[3]	valid_0's auc: 0.834303	valid_0's custom: 0.738318
[4]	valid_0's auc: 0.840909	valid_0's custom: 0.775701
[5]	valid_0's auc: 0.843542	valid_0's custom: 0.780374
[6]	valid_0's auc: 0.844605	valid_0's custom: 0.785047
[7]	valid_0's auc: 0.843865	valid_0's custom: 0.775701
[8]	valid_0's auc: 0.841925	valid_0's custom: 0.803738
[9]	valid_0's auc: 0.842757	valid_0's custom: 0.808411
[10]	valid_0's auc: 0.842849	valid_0's custom: 0.803738
[11]	valid_0's auc: 0.845067	valid_0's custom: 0.808411
[12]	valid_0's auc: 0.84696	valid_0's custom: 0.813084
[13]	valid_0's auc: 0.845297	valid_0's custom: 0.799065
[14]	valid_0's auc: 0.846499	valid_0's custom: 0.799065
[15]	valid_0's auc: 0.843912	valid_0's custom: 0.794393
[16]	valid_0's auc: 0.845621	valid_0's custom: 0.794393
[17]	valid_0's auc: 0.846637	valid_0's custom: 0.799